In [206]:
import numpy as np
import math
import itertools
from functools import cache

In [207]:
n = 10
q = 2
x=np.random.randint(0,q,(n),dtype=np.ubyte)

$D_t(x)$ denotes the $t$ deletion ball of a sequence $x$ of size $n, assuming $t<n$.\
$D_t(x)$ is all of the subsequences of size $t$ of $x$.


In [208]:
def get_deletion_ball(x, t, N=-1):
    if N == -1:
        return np.array(list(set(itertools.combinations(x, x.size-t))))
    else:
        result = set()
        for c in itertools.combinations(x, x.size-t):
            result.add(c)
            if len(result) == N:
                break

        return np.array(list(result))



In [209]:
r=1
D_x_1 = get_deletion_ball(x,r)
print(f'For {x} size {n} vector of alphabet {q} we get a radius {r} deletion ball of size {D_x_1.shape[0]}')

For [0 1 0 1 1 1 1 1 1 1] size 10 vector of alphabet 2 we get a radius 1 deletion ball of size 4


In [210]:
def get_composition_vector(x):
    k = np.zeros(q,dtype=int)
    for i in x:
        k[i] += 1
    return k

def get_ordering(x):
    k = get_composition_vector(x)
    t = np.argsort(-k, kind='stable') # -k so it will be in descending order
    c = k[t]
    return t, c

def number_of_runs(x):
    runs = 1
    prev_x = x[0]
    for i in x[1:]:
        if i != prev_x:
            runs += 1
        prev_x = i

    return runs

@cache
def get_maximal_deletion_ball_size(n, t, q=2):
    if n < t or t < 0:
        return 0
    if q == 1:
        return 1

    size = 0
    for i in range(t+1):
        size += math.comb(n-t, i) * get_maximal_deletion_ball_size(t, t-i, q-1)
    return size

@cache
def get_maximal_insertion_ball_size(n, t, q=2):
    if n < 0 or t < 0:
        return 0
    size = 0
    for i in range(t+1):
        size += math.comb(n+t, i)*int(math.pow((q-1),i))
    return size

@cache
def get_maximal_number_of_common_subsequences(n, t, q=2):
    if n <= t or t <= 0:
        return 0
    return get_maximal_deletion_ball_size(n,t,q) - get_maximal_deletion_ball_size(n-1,t,q) + get_maximal_deletion_ball_size(n-2,t-1,q)

In [211]:
k_x = get_composition_vector(x)
t_x, l_x = get_ordering(x)
print(x)
print(k_x)
print(t_x)
print(l_x)
print(x, number_of_runs(x))
print(get_maximal_deletion_ball_size(n,1))
print(get_maximal_insertion_ball_size(n,1))
print(get_maximal_number_of_common_subsequences(n,1))
print(get_maximal_number_of_common_subsequences(n,2))
print(get_maximal_number_of_common_subsequences(7,3, 3))

[0 1 0 1 1 1 1 1 1 1]
[2 8]
[1 0]
[8 2]
[0 1 0 1 1 1 1 1 1 1] 4
10
12
2
16
24


In [212]:
def get_threshold(n, t, order_comp, q=2):
    for i in range(q):
        w_i = order_comp[i]
        tau_i = get_maximal_number_of_common_subsequences(n-i-1,t-i,q)
        if w_i > tau_i:
            return i

In [213]:
def get_u_a_i(subsequences, a, i):
    if i == 1:
        return subsequences[:,subsequences[0,:] == a][1:]
    results = []
    for s in subsequences.T:
        if len(np.where(s == a)[0]) == 0: continue
        if np.where(s == a)[0][0] == i-1:
            results.append(s)

    if len(results) == 0:
        return None
    
    return np.stack(results).T[i:]
    

The following algorithm reconstructs a sequence $X \in F_q^n$ given $N_q^-(n,t)+1$ different subsequences of $X$ of size $t$\
The algorithm parameters:
* **n**             - The original sequence size to reconstruct.
* **subsequences**  - known subsequences of $X$, this is a ndarray of shape $(t, k)$ where $k$ is at least $N_q^-(n,t)+1$.
* **q**             - Size of the alphabet, the alphabets is assumed to be $[0,1,\dots, q-1]$.
* **verbose**       - control verbosity.




In [214]:
def reconstruct_x_from_subsequences(n, subsequences, q=2, verbose=False):
    t = n-subsequences.shape[0]
    reconstruction = np.array([], dtype=int)
    if verbose:
        print(subsequences.shape)
    while t >= 1:
        order_perm, order_comp = get_ordering(subsequences[0])
        j = get_threshold(n, t, order_comp, q)
        reconstruction = np.concatenate((reconstruction, order_perm[:j+1]))
        if verbose:
            print(j, reconstruction)

        n = n-j-1
        t = t-j    
        N = get_maximal_number_of_common_subsequences(n,t,q)+1
        subsequences = get_u_a_i(subsequences, order_perm[j], 1)
        subsequences = subsequences[:,:N]
        if verbose:
            print(subsequences.shape)
        
    return np.concatenate((reconstruction, subsequences.T[0]))

Let us try and calculate the complexity for q=2

In [215]:
n = 20
t = 2
q = 20


x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_subsequences(n,t,q)+1
print(N)
D_x = get_deletion_ball(x, t, N)

while D_x.shape[0] < N:
    print('1')
    x=np.random.randint(0,q,(n),dtype=np.ubyte)
    D_x = get_deletion_ball(x, t, N)

38


In [216]:
subsequences = D_x.T
# print(subsequences.shape)
# print(get_next_subsequnces(subsequences,0).shape)
# print(get_next_subsequnces(subsequences,0))
reconstructed_x = reconstruct_x_from_subsequences(n, subsequences, q)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))
# print(subsequences[0])

[[14 14 14 11 14 14 11 13 14 14 11 14 11 14 14]
 [13  2 11 13 11 11  2  2 13 11 13 13 13 11 11]
 [ 2 14 13  2 13  2 14 14  2 14  2 14 14 13  2]
 [12 12  2 12 14 14 12 12 14 12 14 12 12 12 12]]
Reconstructed [17  8 15 13  8 17 16  2 16  8  5  2  3  7 14 11 13  2 14 12]
From [17  8 15 13  8 17 16  2 16  8  5  2  3  7 14 11 13  2 14 12]
True


In [ ]:
def get_deletion_ball(x, t, N=-1):
    if N == -1:
        return np.array(list(set(itertools.combinations(x, x.size-t))))
    else:
        result = set()
        for c in itertools.combinations(x, x.size-t):
            result.add(c)
            if len(result) == N:
                break

        return np.array(list(result))


In [218]:
def get_m_vector(subsequences, t, a):
    m_vec = np.zeros((t,), dtype=np.uint)
    for i in range(t):
        u_a_i = get_u_a_i(subsequences, a, i+1)
        if u_a_i is None:
            m_vec[i] = 0
        else:
            m_vec[i] = u_a_i.shape[1]

    return m_vec

In [219]:
get_m_vector(subsequences, 2, 3)

ValueError: need at least one array to stack